In [11]:
# module import
from scipy.io import loadmat
import numpy as np
import os
from glob import glob
import h5py
from GLM.utils import smoothing_penalty_matrix
from GLM.glm import GaussianGLMEstimator
from scipy.io import loadmat
import mat73
import os
import numpy as np
import pandas as pd
from GLM import glm,utils
import dill as pickle
from ChangeOfMind.functions import processing as proc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

import patsy

In [3]:
x0, y0 = 25, 25  # Center of place field
sigma = 5  # Standard deviation of Gaussian
max_rate = 20  # Maximum firing rate (spikes per time step)
# Generate movement trajectory
num_steps = 5000
x_positions = np.random.rand(num_steps)* 50
y_positions = np.random.rand(num_steps)* 50
# Compute firing rate based on 2D Gaussian
def gaussian_firing_rate(x, y, x0, y0, sigma, max_rate):
    return max_rate * np.exp(-((x - x0)**2 + (y - y0)**2) / (2 * sigma**2))
firing_rates = gaussian_firing_rate(x_positions, y_positions, x0, y0, sigma, max_rate)

In [22]:
# specify knots
bins = 10
knots = np.linspace(x_positions.min() + 1, x_positions.max() - 1, bins+1)

In [33]:
# create formula
formulas = []
formulas.append("y~1")
# formulas.append(f"y~te(cr(x_positions,degree=0,knots={knots}), bs(y_positions,degree=0,knots={knots}))")
formulas.append(f"y~te(cr(x_positions,df=7), cr(y_positions,df=7))")

In [35]:
# create model
model = GaussianGLMEstimator()

In [36]:
X_train = pd.DataFrame({"x_positions": x_positions, "y_positions": y_positions})

In [37]:
smooth_lambdas = [0.0001, 0.001, 0.01, 0.1,1.0]

# Set up the parameter grid
param_grid = {
    'formula': formulas,  # Different model formulas
    'smooth_lambda': smooth_lambdas  # Different regularization strengths
}

glm_model = glm.GaussianGLMEstimator()
        
# Use GridSearchCV for cross-validation
grid_search = GridSearchCV(
    estimator=glm_model,
    param_grid=param_grid,
    scoring=make_scorer(glm.gaussian_nll_scorer, greater_is_better=False),  # Use Poisson NLL
    cv=5,  # 5-fold cross-validation
    n_jobs=-1  # Use multiple CPU cores
)

# Fit GridSearchCV on the dataset
grid_search.fit(X_train, firing_rates)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)